# Load packages

In [1]:
import numpy as np
import pandas as pd

# Prep other sources

In [2]:
all_games_df = pd.read_csv('Data/~Created data/all_games_df.csv')
all_games_df = all_games_df[['is_tourney','Season','HTeamID','RTeamID','Hwin']]
all_games_df.head()

,is_tourney,Season,HTeamID,RTeamID,Hwin
0,0.0,2004,1113.0,1168.0,1
1,0.0,2004,1337.0,1168.0,0
2,0.0,2004,1414.0,1168.0,1
3,0.0,2004,1142.0,1168.0,1
4,0.0,2004,1429.0,1168.0,1


In [3]:
kp_df = pd.read_csv('Data/~Created data/kp_all.csv')
kp_df.head()

,team,conf,adjem,adjo,adjd,luck,Season,TeamID
0,virginia,ACC,35.66,123.6,87.9,0.024,2019,1438.0
1,gonzaga,WCC,32.79,125.1,92.3,0.008,2019,1211.0
2,duke,ACC,31.99,120.1,88.1,0.018,2019,1181.0
3,michigan st.,B10,31.36,121.7,90.4,-0.007,2019,1277.0
4,michigan,B10,29.44,115.5,86.1,-0.014,2019,1276.0


In [4]:
regseason_df = pd.read_csv('Data/~Created data/regseason_df.csv')
regseason_df  = regseason_df[['TeamID','Season','wins_top25','PointMargin','FG','FG3']]
regseason_df.head()

,TeamID,Season,wins_top25,PointMargin,FG,FG3
0,1101,2014,0,-15.476190,0.410184,0.380275
1,1101,2015,0,-10.857143,0.407501,0.381507
2,1101,2016,0,-6.407407,0.442606,0.364517
3,1101,2017,0,-4.760000,0.457867,0.371026
4,1101,2018,0,-2.148148,0.452366,0.327240


In [5]:
massey_df = pd.read_csv('Data/Kaggle NCAA/MasseyOrdinals_thru_2019_day_128.csv')
POM_df = massey_df[massey_df['SystemName'].str.contains("POM")]
POM_end_df = POM_df.loc[POM_df['RankingDayNum'] == 128]
POM_end_df.rename(columns={'OrdinalRank': 'RankPOM'}, inplace=True)
POM_end_df = POM_end_df[['Season','TeamID','RankPOM']]
POM_end_df.head()

C:\Users\dlorenz\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Season,TeamID,RankPOM
392714,2005,1102,54
392715,2005,1103,75
392716,2005,1104,18
392717,2005,1105,291
392718,2005,1106,300


# Create test set

In [6]:
#Test set (this sets the data up in the format Kaggle needs for scoring)
df_seeds = pd.read_csv('Data/Kaggle NCAA/NCAATourneySeeds.csv')
df_seeds = df_seeds[df_seeds['Season']==2019]
df_19_tourney = df_seeds.merge(df_seeds, how='inner', on='Season')
df_19_tourney = df_19_tourney[df_19_tourney['TeamID_x'] < df_19_tourney['TeamID_y']]
df_19_tourney['ID'] = df_19_tourney['Season'].astype(str) + '_' \
              + df_19_tourney['TeamID_x'].astype(str) + '_' \
              + df_19_tourney['TeamID_y'].astype(str)
df_19_tourney['SeedInt_x'] = [int(x[1:3]) for x in df_19_tourney['Seed_x']]
df_19_tourney['SeedInt_y'] = [int(x[1:3]) for x in df_19_tourney['Seed_y']]

#Make home team lower seed (consistent with training data)
df_19_tourney.loc[df_19_tourney['SeedInt_x']<df_19_tourney['SeedInt_y'], 'HTeamID'] = df_19_tourney['TeamID_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']>df_19_tourney['SeedInt_y'], 'HTeamID'] = df_19_tourney['TeamID_y']
df_19_tourney.loc[df_19_tourney['SeedInt_x']<df_19_tourney['SeedInt_y'], 'RTeamID'] = df_19_tourney['TeamID_y']
df_19_tourney.loc[df_19_tourney['SeedInt_x']>df_19_tourney['SeedInt_y'], 'RTeamID'] = df_19_tourney['TeamID_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']==df_19_tourney['SeedInt_y'], 'HTeamID'] = df_19_tourney['TeamID_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']==df_19_tourney['SeedInt_y'], 'RTeamID'] = df_19_tourney['TeamID_y']

df_19_tourney.loc[df_19_tourney['SeedInt_x']<df_19_tourney['SeedInt_y'], 'HSeed'] = df_19_tourney['SeedInt_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']>df_19_tourney['SeedInt_y'], 'HSeed'] = df_19_tourney['SeedInt_y']
df_19_tourney.loc[df_19_tourney['SeedInt_x']<df_19_tourney['SeedInt_y'], 'RSeed'] = df_19_tourney['SeedInt_y']
df_19_tourney.loc[df_19_tourney['SeedInt_x']>df_19_tourney['SeedInt_y'], 'RSeed'] = df_19_tourney['SeedInt_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']==df_19_tourney['SeedInt_y'], 'HSeed'] = df_19_tourney['SeedInt_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']==df_19_tourney['SeedInt_y'], 'RSeed'] = df_19_tourney['SeedInt_y']

df_19_tourney['is_tourney'] = 1

df_19_tourney = df_19_tourney.drop(['Seed_x','Seed_y','TeamID_x','TeamID_y','SeedInt_x','SeedInt_y'], axis=1)
df_19_tourney.sort_index()

,Season,ID,HTeamID,RTeamID,HSeed,RSeed,is_tourney
1,2019,2019_1181_1277,1181.0,1277.0,1.0,2.0,1
2,2019,2019_1181_1261,1181.0,1261.0,1.0,3.0,1
3,2019,2019_1181_1439,1181.0,1439.0,1.0,4.0,1
4,2019,2019_1181_1280,1181.0,1280.0,1.0,5.0,1
5,2019,2019_1181_1268,1181.0,1268.0,1.0,6.0,1
6,2019,2019_1181_1257,1181.0,1257.0,1.0,7.0,1
7,2019,2019_1181_1433,1181.0,1433.0,1.0,8.0,1
8,2019,2019_1181_1416,1181.0,1416.0,1.0,9.0,1
9,2019,2019_1181_1278,1181.0,1278.0,1.0,10.0,1
11,2019,2019_1181_1396,1181.0,1396.0,1.0,11.0,1


In [7]:
home_road = ['H','R']
for hr in home_road:
    df_19_tourney = pd.merge(df_19_tourney, regseason_df, left_on=['Season',hr+'TeamID'], right_on = ['Season','TeamID'], how='left')
    df_19_tourney.rename(columns={'wins_top25': hr+'wins_top25'}, inplace=True)
    df_19_tourney.rename(columns={'PointMargin': hr+'PointMargin'}, inplace=True)
    df_19_tourney.rename(columns={'FG': hr+'FG'}, inplace=True)
    df_19_tourney.rename(columns={'FG3': hr+'FG3'}, inplace=True)
    df_19_tourney = df_19_tourney.drop(['TeamID'], axis=1)

In [8]:
for hr in home_road:
    df_19_tourney = pd.merge(df_19_tourney, POM_end_df, left_on=['Season',hr+'TeamID'], right_on = ['Season','TeamID'], how='left')
    df_19_tourney.rename(columns={'RankPOM': hr+'RankPOM'}, inplace=True)
    df_19_tourney = df_19_tourney.drop(['TeamID'], axis=1)

In [9]:
efficiency_list = ['conf','adjem','adjo','adjd','luck','TeamID']
for hr in home_road:
    df_19_tourney = pd.merge(df_19_tourney, kp_df, left_on=[hr+'TeamID','Season'], right_on = ['TeamID','Season'], how='inner')
    df_19_tourney = df_19_tourney.drop(['TeamID'], axis=1)
    for metric in efficiency_list:
        df_19_tourney.rename(columns={metric: hr+metric}, inplace=True)
    if hr == 'H':
        df_19_tourney.rename(columns={'team': 'home'}, inplace=True)
    if hr == 'R':
        df_19_tourney.rename(columns={'team': 'road'}, inplace=True) 

In [10]:
df_19_tourney['Htourny20plus'] = 0
df_19_tourney['Rtourny20plus'] = 0

experienced_teams = ['kansas','north carolina','kentucky','duke','michigan st.','wisconsin','florida','villanova','gonzaga','louisville','arizona','xavier','connecticut','syracuse','butler','ohio st.','ucla','west virginia','texas','michigan','pittsburgh','memphis','oregon']
for team in experienced_teams:
    df_19_tourney.loc[df_19_tourney['home']==team, 'Htourny20plus'] = 1
    df_19_tourney.loc[df_19_tourney['road']==team, 'Rtourny20plus'] = 1

In [11]:
df_19_tourney['HBig4Conf'] = 0
df_19_tourney['RBig4Conf'] = 0
conferences = ['ACC','B10','B12','SEC']
for conf in conferences:
    df_19_tourney.loc[df_19_tourney['Hconf']==conf, 'HBig4Conf'] = 1
    df_19_tourney.loc[df_19_tourney['Rconf']==conf, 'RBig4Conf'] = 1

In [12]:
list(df_19_tourney)

['Season',
 'ID',
 'HTeamID',
 'RTeamID',
 'HSeed',
 'RSeed',
 'is_tourney',
 'Hwins_top25',
 'HPointMargin',
 'HFG',
 'HFG3',
 'Rwins_top25',
 'RPointMargin',
 'RFG',
 'RFG3',
 'HRankPOM',
 'RRankPOM',
 'home',
 'Hconf',
 'Hadjem',
 'Hadjo',
 'Hadjd',
 'Hluck',
 'road',
 'Rconf',
 'Radjem',
 'Radjo',
 'Radjd',
 'Rluck',
 'Htourny20plus',
 'Rtourny20plus',
 'HBig4Conf',
 'RBig4Conf']

In [13]:
df_19_tourney.tail()

,Season,ID,HTeamID,RTeamID,HSeed,RSeed,is_tourney,Hwins_top25,HPointMargin,HFG,...,road,Rconf,Radjem,Radjo,Radjd,Rluck,Htourny20plus,Rtourny20plus,HBig4Conf,RBig4Conf
2273,2019,2019_1159_1330,1330.0,1159.0,14.0,15.0,1,0,5.323529,0.411639,...,colgate,Pat,4.29,110.0,105.8,0.021,0,0,0,0
2274,2019,2019_1124_1159,1124.0,1159.0,9.0,15.0,1,3,4.468750,0.442081,...,colgate,Pat,4.29,110.0,105.8,0.021,0,0,1,0
2275,2019,2019_1113_1159,1113.0,1159.0,11.0,15.0,1,2,4.750000,0.450937,...,colgate,Pat,4.29,110.0,105.8,0.021,0,0,0,0
2276,2019,2019_1120_1159,1120.0,1159.0,5.0,15.0,1,3,10.294118,0.447579,...,colgate,Pat,4.29,110.0,105.8,0.021,0,0,1,0
2277,2019,2019_1101_1159,1101.0,1159.0,15.0,15.0,1,0,6.827586,0.458861,...,colgate,Pat,4.29,110.0,105.8,0.021,0,0,0,0


In [14]:
df_19_tourney = df_19_tourney.fillna(df_19_tourney.mean())

# Output to csv

In [15]:
df_19_tourney.to_csv('Data/~Created data/test_combos_df_19.csv', index=False)